In [7]:
import polars as pl
import pandas as pd
import numpy as np
import math as math
# find a better way to import?
import sys
import os
sys.path.append(os.path.join(os.path.dirname(''), '..'))

from constants import *


In [8]:
# read all data
players = pl.read_csv('../nfl-big-data-bowl-2024/players.csv')
plays = pl.read_csv('../nfl-big-data-bowl-2024/plays.csv',infer_schema_length=100000)
games = pl.read_csv('../nfl-big-data-bowl-2024/games.csv',infer_schema_length=10000)
tracking = pl.read_csv('../nfl-big-data-bowl-2024/tracking_week*.csv',infer_schema_length=10000)

In [9]:
# normalize data
players = players.with_columns([pl.col('nflId').cast(str)])
plays = plays.join(games,on='gameId')
plays = plays.with_columns([
    (pl.col('gameId').cast(str) + '-'
     + pl.col('playId').cast(str)).alias('uniquePlayId')
])

tracking = tracking.with_columns(
    (pl.col('gameId').cast(str) + '-'
     + pl.col('playId').cast(str)).alias('uniquePlayId'),
    (pl.col('gameId').cast(str) + '-'
     + pl.col('playId').cast(str) + '-'
     + pl.col('nflId').cast(str)).alias('uniquePlayerId'),
)

# normalize position
tracking=tracking.with_columns([
    pl.when(pl.col('playDirection')=='right').then(53.3-pl.col('y')).otherwise(pl.col('y')).alias('adjustedX'),
    pl.when(pl.col('playDirection')=='right').then(pl.col('x')).otherwise(120-pl.col('x')).alias('adjustedY')
])

tracking=tracking.with_columns([
    pl.when(pl.col('event')=='ball_snap').then(pl.col('frameId')).otherwise(-1).alias('startingFrameId'),
])
tracking=tracking.with_columns([
    pl.col('startingFrameId').max().over(pl.col('uniquePlayId')),
])
tracking=tracking.with_columns([
    (pl.col('frameId') - pl.col('startingFrameId')).alias('framesSinceSnap'),
])

# normalize orientation 'o' and direction 'dir'
# convert 'NA' to 0
replacement_values = {'NA': '0'}
tracking = tracking.with_columns(
    pl.col('o').apply(lambda x: replacement_values.get(x, x)),
    pl.col('dir').apply(lambda x: replacement_values.get(x, x)),
)

tracking=tracking.with_columns([
    pl.when(pl.col('playDirection')=='right').then(pl.col('dir').cast(pl.Float64)).otherwise(180-pl.col('dir').cast(pl.Float64)).alias('adjustedDir'),
    pl.when(pl.col('playDirection')=='right').then(pl.col('o').cast(pl.Float64)).otherwise(180-pl.col('o').cast(pl.Float64)).alias('adjustedO'),
])

In [27]:
CONE_ANGLE = 15 # degrees
MAX_DISTANCE = 5 # feet
BLOCKING_RADIUS = 1 # feet

def looking_to_block_or_blocking(player1: pl.DataFrame, player2: pl.DataFrame) -> int:
    if is_blocking(player1, player2):
        return 2

    if is_in_vision_cone(player1, player2):
        return 1

    return 0

def is_in_vision_cone(player1: pl.DataFrame, player2: pl.DataFrame) -> bool:
    return (is_in_angle(player1, player2) and is_in_distance(player1, player2))

def is_in_angle(player1: pl.DataFrame, player2: pl.DataFrame) -> bool:
    half_cone_angle = CONE_ANGLE / 2

    y_dist = player2.select(pl.col('adjustedY')).item() - player1.select(pl.col('adjustedY')).item()
    x_dist = player2.select(pl.col('adjustedX')).item() - player1.select(pl.col('adjustedX')).item()

    player1_orientation = player1.select(pl.col('adjustedO')).item()

    angle = math.degrees(math.atan2(y_dist, x_dist))

    if player1_orientation - half_cone_angle <= angle <= player1_orientation + half_cone_angle:
        return True
    
    return False

def is_in_distance(player1: pl.DataFrame, player2: pl.DataFrame) -> bool:
    distance_between_players = calculate_distance(player1, player2)
    print(distance_between_players)
    if distance_between_players <= MAX_DISTANCE:
        return True
    
    return False

def is_blocking(player1: pl.DataFrame, player2: pl.DataFrame) -> bool:
    distance_between_players = calculate_distance(player1, player2)
    if distance_between_players <= BLOCKING_RADIUS:
        return True

    return False


def calculate_distance(player1: pl.DataFrame, player2: pl.DataFrame) -> float:
    y_dist = abs(player1.select(pl.col('adjustedY')).item() - player2.select(pl.col('adjustedY')).item())
    x_dist = abs(player1.select(pl.col('adjustedX')).item() - player2.select(pl.col('adjustedX')).item())

    return math.sqrt(x_dist**2 + y_dist**2)

In [31]:
gameId = 2022090800
playId = 56
frameId = 1

current_game_play_frame = (
    tracking.filter(pl.col('gameId')==gameId)
        .filter(pl.col('playId')==playId)
        .filter(pl.col('frameId')==frameId)
)

# validating :)
data = {
    'adjustedX': [50.00, 50.00],
    'adjustedY': [49.00, 50.00],
    'adjustedO': [90.00, 270.00]
}

test_df = pl.DataFrame(data)

player1 = test_df[0]
player2 = test_df[1]

looking_to_block_or_blocking(player1, player2)

2